In [1]:
import math
import datetime
import warnings
import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
from tabulate import tabulate
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
df = pd.read_csv(r'C:\Users\ws-1\Downloads\Study.csv')
df.head()

,Study_number,Therapeutic_Area,planned_start_date,Study Actual End Date,Study Actual Start Date,planned_end_date,Study Enrolled Subjects,Study Planned Subject,Study Drop-Outs
0,AJ_4567,Bariatrics,12/30/2005,7/13/2014,2/14/2006,11/4/2012,7801,8232,431
1,AT_2687,Addiction Medicine,10/15/2000,4/22/2008,10/19/2000,12/2/2006,6395,6665,270
2,BN_5333,Infectious Disease,4/24/2012,11/22/2020,8/27/2012,7/31/2019,6478,6877,399
3,CU_8814,Pulmonary Medicine,3/10/2014,1/4/2021,7/13/2014,6/30/2019,4929,5196,267
4,DP_5759,Pulmonary Medicine,9/13/2010,4/11/2018,10/31/2010,9/14/2016,7559,7991,432


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Study_number             50 non-null     object
 1   Therapeutic_Area         50 non-null     object
 2   planned_start_date       50 non-null     object
 3   Study Actual End Date    50 non-null     object
 4   Study Actual Start Date  50 non-null     object
 5   planned_end_date         50 non-null     object
 6   Study Enrolled Subjects  50 non-null     int64 
 7   Study Planned Subject    50 non-null     int64 
 8   Study Drop-Outs          50 non-null     int64 
dtypes: int64(3), object(6)
memory usage: 3.6+ KB


In [4]:
df['planned_start_date'] = pd.to_datetime(df['planned_start_date'],errors='coerce')
df['planned_end_date'] = pd.to_datetime(df['planned_end_date'],errors='coerce')
df['Study Actual Start Date'] = pd.to_datetime(df['Study Actual Start Date'],errors='coerce')
df['Study Actual End Date'] = pd.to_datetime(df['Study Actual End Date'],errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Study_number             50 non-null     object        
 1   Therapeutic_Area         50 non-null     object        
 2   planned_start_date       50 non-null     datetime64[ns]
 3   Study Actual End Date    50 non-null     datetime64[ns]
 4   Study Actual Start Date  50 non-null     datetime64[ns]
 5   planned_end_date         50 non-null     datetime64[ns]
 6   Study Enrolled Subjects  50 non-null     int64         
 7   Study Planned Subject    50 non-null     int64         
 8   Study Drop-Outs          50 non-null     int64         
dtypes: datetime64[ns](4), int64(3), object(2)
memory usage: 3.6+ KB


In [5]:
df=df.drop(['Study_number','Therapeutic_Area',  ],axis=1)
df.head()

,planned_start_date,Study Actual End Date,Study Actual Start Date,planned_end_date,Study Enrolled Subjects,Study Planned Subject,Study Drop-Outs
0,2005-12-30,2014-07-13,2006-02-14,2012-11-04,7801,8232,431
1,2000-10-15,2008-04-22,2000-10-19,2006-12-02,6395,6665,270
2,2012-04-24,2020-11-22,2012-08-27,2019-07-31,6478,6877,399
3,2014-03-10,2021-01-04,2014-07-13,2019-06-30,4929,5196,267
4,2010-09-13,2018-04-11,2010-10-31,2016-09-14,7559,7991,432


In [6]:
date_columns = ['planned_start_date', 'planned_end_date', 'Study Actual Start Date', 'Study Actual End Date']

# Convert string dates to datetime objects
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')  # Convert to datetime, handle errors gracefully

# Define the function to convert dates to ordinals
def date_to_ordinal(date):
    return date.toordinal() if pd.notnull(date) else np.nan

# Apply the date_to_ordinal function to each date column
for col in date_columns:
    df[f'{col}_num'] = df[col].map(date_to_ordinal)

# Optionally, drop the original datetime columns if not needed
df = df.drop(columns=date_columns)

# Display the resulting DataFrame
print(df.head())

   Study Enrolled Subjects  Study Planned Subject  Study Drop-Outs  \
0                     7801                   8232              431   
1                     6395                   6665              270   
2                     6478                   6877              399   
3                     4929                   5196              267   
4                     7559                   7991              432   

   planned_start_date_num  planned_end_date_num  Study Actual Start Date_num  \
0                  732310                734811                       732356   
1                  730408                732647                       730412   
2                  734617                737271                       734742   
3                  735302                737240                       735427   
4                  734028                736221                       734076   

   Study Actual End Date_num  
0                     735427  
1                     733154  
2    

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Study Enrolled Subjects      50 non-null     int64
 1   Study Planned Subject        50 non-null     int64
 2   Study Drop-Outs              50 non-null     int64
 3   planned_start_date_num       50 non-null     int64
 4   planned_end_date_num         50 non-null     int64
 5   Study Actual Start Date_num  50 non-null     int64
 6   Study Actual End Date_num    50 non-null     int64
dtypes: int64(7)
memory usage: 2.9 KB


In [8]:
# Define feature and target columns
X = df[['planned_end_date_num', 'planned_start_date_num', 'Study Actual Start Date_num','Study Enrolled Subjects', 'Study Planned Subject','Study Drop-Outs']]
y = df['Study Actual End Date_num']

In [9]:
# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [10]:
# Output the shapes of the resulting splits
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (40, 6)
X_test shape: (10, 6)
y_train shape: (40,)
y_test shape: (10,)


In [11]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Initialize the XGBoost Regressor
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Train the model
xg_reg.fit(X_train, y_train)
# Predict on the train and test sets
y_train_pred = xg_reg.predict(X_train)
y_test_pred = xg_reg.predict(X_test)
# Calculate metrics for the training set
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
train_r2 = r2_score(y_train, y_train_pred)

# Calculate metrics for the test set
test_mae = mean_absolute_error(y_test, y_test_pred)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_r2 = r2_score(y_test, y_test_pred)

# Display the results
print("Training Set Metrics:")
print(f"Mean Absolute Error (MAE): {train_mae}")
print(f"Root Mean Squared Error (RMSE): {train_rmse}")
print(f"R^2 Score: {train_r2}")

print("\nTest Set Metrics:")
print(f"Mean Absolute Error (MAE): {test_mae}")
print(f"Root Mean Squared Error (RMSE): {test_rmse}")
print(f"R^2 Score: {test_r2}")


Training Set Metrics:
Mean Absolute Error (MAE): 0.090625
Root Mean Squared Error (RMSE): 0.10643366478704
R^2 Score: 0.9999999972353111

Test Set Metrics:
Mean Absolute Error (MAE): 367.98125
Root Mean Squared Error (RMSE): 595.5928240915727
R^2 Score: 0.8980497718393218


In [12]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize the AdaBoost Regressor
ada_reg = AdaBoostRegressor(n_estimators=100, random_state=42)

# Train the AdaBoost model
ada_reg.fit(X_train, y_train)

# Predict on the train and test sets
y_train_pred = ada_reg.predict(X_train)
y_test_pred = ada_reg.predict(X_test)

# Calculate metrics for the training set
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
train_r2 = r2_score(y_train, y_train_pred)

# Calculate metrics for the test set
test_mae = mean_absolute_error(y_test, y_test_pred)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_r2 = r2_score(y_test, y_test_pred)

# Display the results
print("Training Set Metrics:")
print(f"Mean Absolute Error (MAE): {train_mae}")
print(f"Root Mean Squared Error (RMSE): {train_rmse}")
print(f"R^2 Score: {train_r2}")

print("\nTest Set Metrics:")
print(f"Mean Absolute Error (MAE): {test_mae}")
print(f"Root Mean Squared Error (RMSE): {test_rmse}")
print(f"R^2 Score: {test_r2}")


Training Set Metrics:
Mean Absolute Error (MAE): 77.97073482073901
Root Mean Squared Error (RMSE): 98.04259981635353
R^2 Score: 0.9976540547068905

Test Set Metrics:
Mean Absolute Error (MAE): 380.9621212121216
Root Mean Squared Error (RMSE): 618.7117143176503
R^2 Score: 0.8899814374348345


In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize the Random Forest Regressor
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_reg.fit(X_train, y_train)

# Predict on the train and test sets
y_train_pred = rf_reg.predict(X_train)
y_test_pred = rf_reg.predict(X_test)

# Calculate metrics for the training set
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
train_r2 = r2_score(y_train, y_train_pred)

# Calculate metrics for the test set
test_mae = mean_absolute_error(y_test, y_test_pred)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_r2 = r2_score(y_test, y_test_pred)

# Display the results
print("Training Set Metrics:")
print(f"Mean Absolute Error (MAE): {train_mae}")
print(f"Root Mean Squared Error (RMSE): {train_rmse}")
print(f"R^2 Score: {train_r2}")

print("\nTest Set Metrics:")
print(f"Mean Absolute Error (MAE): {test_mae}")
print(f"Root Mean Squared Error (RMSE): {test_rmse}")
print(f"R^2 Score: {test_r2}")


Training Set Metrics:
Mean Absolute Error (MAE): 92.6077500000014
Root Mean Squared Error (RMSE): 115.4729916365706
R^2 Score: 0.9967457637593478

Test Set Metrics:
Mean Absolute Error (MAE): 381.39000000001397
Root Mean Squared Error (RMSE): 616.5561559987908
R^2 Score: 0.8907466995460294


In [14]:
import joblib

# Save the model
joblib.dump(xg_reg, 'xgboost_model_study.pkl')

['xgboost_model_study.pkl']

In [15]:
# Dump the trained model to a file
joblib.dump(rf_reg, 'random_forest_model.pkl')

['random_forest_model.pkl']

In [16]:
import joblib

# Dump the AdaBoost model
joblib.dump(ada_reg, 'adaboost_model.pkl')


['adaboost_model.pkl']

In [19]:
import joblib
import pandas as pd
import numpy as np

# Load the trained models
rf_reg = joblib.load('random_forest_model.pkl')
xg_reg = joblib.load('xgboost_model_study.pkl')
ada_reg = joblib.load('adaboost_model.pkl')

def date_to_ordinal(date_str):
    try:
        return pd.to_datetime(date_str).toordinal()
    except ValueError:
        return np.nan

def predict_end_date(planned_end_date, planned_start_date, actual_start_date, enrolled_subjects, planned_subjects, drop_outs, model):
    # Convert input dates to ordinal
    data = {
        'planned_end_date_num': [date_to_ordinal(planned_end_date)],
        'planned_start_date_num': [date_to_ordinal(planned_start_date)],
        'Study Actual Start Date_num': [date_to_ordinal(actual_start_date)],
        'Study Enrolled Subjects': [enrolled_subjects],
        'Study Planned Subject': [planned_subjects],
        'Study Drop-Outs': [drop_outs]
    }
    input_df = pd.DataFrame(data)
    
    # Make prediction
    predicted_ordinal = model.predict(input_df)[0]
    
    # Convert predicted ordinal back to date
    predicted_date = pd.to_datetime(pd.Timestamp.fromordinal(int(predicted_ordinal)))
    return predicted_date

def main():
    print("Please enter the following details:")
    planned_end_date = input("Planned End Date (YYYY-MM-DD): ")
    planned_start_date = input("Planned Start Date (YYYY-MM-DD): ")
    actual_start_date = input("Study Actual Start Date (YYYY-MM-DD): ")
    enrolled_subjects = int(input("Study Enrolled Subjects: "))
    planned_subjects = int(input("Study Planned Subjects: "))
    drop_outs = int(input("Study Drop-Outs: "))
    
    # Predict and display the results
    predicted_end_date_rf = predict_end_date(planned_end_date, planned_start_date, actual_start_date, enrolled_subjects, planned_subjects, drop_outs, rf_reg)
    predicted_end_date_xgb = predict_end_date(planned_end_date, planned_start_date, actual_start_date, enrolled_subjects, planned_subjects, drop_outs, xg_reg)
    predicted_end_date_ada = predict_end_date(planned_end_date, planned_start_date, actual_start_date, enrolled_subjects, planned_subjects, drop_outs, ada_reg)
    
    print(f"\nPredicted Study Actual End Date (Random Forest): {predicted_end_date_rf.strftime('%Y-%m-%d')}")
    print(f"Predicted Study Actual End Date (XGBoost): {predicted_end_date_xgb.strftime('%Y-%m-%d')}")
    print(f"Predicted Study Actual End Date (AdaBoost): {predicted_end_date_ada.strftime('%Y-%m-%d')}")

if __name__ == "__main__":
    main()


Please enter the following details:


Planned End Date (YYYY-MM-DD):  2022-01-08
Planned Start Date (YYYY-MM-DD):  2017-06-14
Study Actual Start Date (YYYY-MM-DD):  2017-06-14
Study Enrolled Subjects:  4550
Study Planned Subjects:  4858
Study Drop-Outs:  308



Predicted Study Actual End Date (Random Forest): 2024-03-31
Predicted Study Actual End Date (XGBoost): 2024-01-01
Predicted Study Actual End Date (AdaBoost): 2024-02-05


In [18]:
month/date

NameError: name 'mm' is not defined